Import libraries

In [ ]:
import os
import sys
import glob
import zipfile
from zipfile import BadZipFile
import datetime
import pandas as pd
import hyp3_sdk as sdk
from hyp3_sdk import asf_search
from tqdm.notebook import trange, tqdm
# from tqdm.auto import tqdm

Enter Hyp3 credentials

In [ ]:
hyp3 = sdk.HyP3(prompt=True)
my_info = hyp3.my_info()
print(f"Remaining Quota: {my_info['quota']['remaining']}")

Create download list

In [ ]:
# Project name
# project_name = 'ifg_a_p81_f1048_20x4'
project_name = 'p81f1048_10x2_2'

# Set date bounds
start_date = datetime.datetime(2017, 12, 31)
end_date = datetime.datetime(2021, 12, 31)

# Read in 
df = pd.read_csv('slcs_asc_path81_frame1048.csv',usecols=['Granule Name', 'Acquisition Date'])

# Convert and limit by date
df['Acquisition Date'] = pd.to_datetime(df['Acquisition Date'], format='%Y-%m-%dT%H:%M:%S')
df = df.loc[(df['Acquisition Date']>=start_date) & (df['Acquisition Date']<=end_date)]
df = df.sort_values('Acquisition Date')

granules = list(df['Granule Name'])
print(f'Will create interferograms for {len(granules)} SLCs')

Request Ifg generation

In [ ]:
insar_jobs = sdk.Batch()
first = True
for reference in tqdm(granules):
    neighbors_metadata = asf_search.get_nearest_neighbors(reference, max_neighbors=3)
    for secondary_metadata in neighbors_metadata:
        secondary = secondary_metadata['granuleName']
        if first:
            insar_jobs += hyp3.submit_insar_job(reference, secondary, name=project_name, include_inc_map=True,
                                                include_dem=True, looks='10x2', include_wrapped_phase=False)
            first = False
        else:
            insar_jobs += hyp3.submit_insar_job(reference, secondary, name=project_name, looks='10x2', include_wrapped_phase=False)

Check progress and download

In [ ]:
batch = hyp3.find_jobs(name=project_name)

if batch.complete():
    #filter to only succeeded jobs
    succeeded_jobs = batch.filter_jobs(succeeded=True, running=False, failed=False)

    #download files if not downloaded already
    for job in succeeded_jobs.jobs:
        filename = job.to_dict()['files'][0]['filename']
        location = os.path.join(project_name,filename)
        if not os.path.exists(location):
            job.download_files(location=project_name,create=True)
        
    # #download files if not downloaded already
    # file_list = succeeded_jobs.download_files(location=project_name,create=True)
else:
    #to get updated information
    batch = hyp3.refresh(batch)
    #or to wait until completion and get updated information (which will take a fair bit)
    batch = hyp3.watch(batch)

Unzip files and clip to same extent

In [ ]:
os.chdir('/mnt/i/hyp3_downloads')

In [ ]:
#filter to only succeeded jobs
succeeded_jobs = batch.filter_jobs(succeeded=True, running=False, failed=False)

#download files if not downloaded already
for job in succeeded_jobs.jobs:
    filename = job.to_dict()['files'][0]['filename']
    location = os.path.join(project_name,filename)
    if not os.path.exists(location):
        job.download_files(location=project_name,create=True)

In [ ]:
# project_name = 'tongariro_asc'

zips = glob.glob(os.path.join(project_name,'*.zip'))
unws = [[x,f'{os.path.join(x.split(".")[0],os.path.basename(x).split(".")[0])}_unw_phase.tif'] for x in zips]

zips = [x[0] for x in unws if not os.path.exists(x[1])]
folders = [x.split('.')[0] for x in zips]

bad_files=[]

for zip_file, folder in tqdm(zip(zips, folders), total=len(zips)):
    try:
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            zip_ref.extractall(project_name)
    except BadZipFile:
        bad_files.append(os.path.basename(zip_file))

if len(bad_files) > 0:
    print(f'These files were invalid zips:\n{bad_files}')

In [ ]:
# folders = glob.glob(os.path.join(project_name,'S1*'))
# all_images = []
# for suffix in ['dem','inc_map','water_map','unw_phase','corr']:
#     all_images += [os.path.join(x,f'{os.path.basename(x)}_{suffix}.tif') for x in folders]

# exists = [x for x in all_images if os.path.exists(x)]
# to_clip = [x for x in exists if not os.path.exists(f'{x[:-4]}_clip.tif')]

# cutGeotiffs.cutFiles(to_clip)

Done!